In [1]:
import pandas as pd
import numpy as np

df_con = pd.read_csv("../data/1. 상장폐지종목 정리끝.csv",index_col=0)
df_invis = pd.read_csv("../data/1. 코스닥 기업 개별재무제표.csv",encoding='cp949')

In [2]:
#컬럼명 일치를 위한 컬럼명 변경 
df_con.columns = df_con.columns.str.replace('연결','')

# 빈 값을 넣기 위한 list 생성
list = ['[제조]총자본증가율(IFRS)', '[제조]유형자산증가율(IFRS)', '[제조]비유동자산증가율(IFRS)',
       '[제조]유동자산증가율(IFRS)', '[제조]재고자산증가율(IFRS)', '[제조]자기자본증가율(IFRS)',
       '[제조]매출액증가율(IFRS)', '[제조]순이익증가율(IFRS)', '[제조]매출액총이익률(IFRS)',
       '[제조]매출액순이익률(IFRS)', '[제조]자기자본순이익률(IFRS)', '[제조]총자본사업이익률(IFRS)',
       '[제조]총자본순이익률(IFRS)', '[제조]1주당매출액(IFRS)', '[제조]1주당순이익(IFRS)',
       '[제조]유동비율(IFRS)', '[제조]당좌비율(IFRS)', '[제조]현금비율(IFRS)',
       '[제조]순운전자본비율(IFRS)', '[제조]부채비율(IFRS)', '[제조]이자보상배율(이자비용)(IFRS)',
       '[제조]비유동비율(IFRS)', '[제조]유동자산구성비율(IFRS)', '[제조]자기자본구성비율(IFRS)',
       '[제조]자기자본배율(IFRS)', '[제조]총자본회전률(IFRS)', '[제조]자기자본회전률(IFRS)',
       '[제조]비유동자산회전률(IFRS)', '[제조]재고자산회전률(IFRS)', '[제조]매출채권회전률(IFRS)',
       '[제조]매입채무회전률(IFRS)', '[제조]매입채무회전기간(IFRS)', '[제조]재고자산회전기간(IFRS)',
       '[제조]매출채권회전기간(IFRS)', '[제조]1회전기간(IFRS)', '[제조]운전자본회전률(IFRS)',
       '[제조]순운전자본회전률(IFRS)', '[제조]유형자산회전율(IFRS)', '[제조]유동자산회전률(IFRS)',
       '[제조]당좌자산회전률(IFRS)', '[제조]부가가치(IFRS)', '[제조]총자본투자효율(IFRS)',
       '[제조]설비투자효율(IFRS)', '[제조]기계투자효율(IFRS)', '[제조]부가가치율(IFRS)',
       '[제조]노동소득분배율(IFRS)', '[제조]자본분배율(IFRS)', '[제조]이윤분배율(IFRS)']

# 빈공간을 개별로 넣기 
for i in list:
    df_con[i] = df_con[i].fillna(df_invis[i])

# 개별재무제표가 없는 데이터의 인덱스를 추출해서 없애기
null_growth_idx = df_con[df_con["[제조]총자본증가율(IFRS)"].isnull()].index
df_con.drop(null_growth_idx, inplace=True)

# 추가 다른 컬럼에서 결측치가 있는 데이터 삭제 
null_profit_ratio_idx = df_con[df_con["[제조]이윤분배율(IFRS)"].isnull()].index
df_con.drop(null_profit_ratio_idx, inplace=True)

# 인덱스 초기화
df_con.reset_index(drop='index', inplace=True)

# 상장폐지일이 존재하면서 소속코드가 5인 데이터들의 상장폐지 일 내용 삭제
delisted_data = df_con[~df_con["상장폐지일"].isnull()]
indexes = delisted_data[delisted_data["소속코드"] == 5].index

# replace로 대체
for i in indexes:
    df_con["상장폐지일"] = df_con["상장폐지일"].replace(df_con.loc[i, "상장폐지일"], np.NAN)

print('총 기업수 :',len(df_con["회사명"].unique()))
print('정상기업 :',len(df_con[df_con["소속코드"]==5]["회사명"].unique()))
print('상장폐지 기업수(중복 미포함) :',len(df_con[df_con["소속코드"]==4]["회사명"].unique()))

총 기업수 : 1439
정상기업 : 1204
상장폐지 기업수(중복 미포함) : 178


In [3]:
df_con.columns = df_con.columns.str.replace("제조",'')
df_con.columns = df_con.columns.str.replace('[','')
df_con.columns = df_con.columns.str.replace(']','')
df_con.columns = df_con.columns.str.replace('IFRS','')
df_con.columns = df_con.columns.str.replace("연결",'')
df_con.columns = df_con.columns.str.replace("(",'')
df_con.columns = df_con.columns.str.replace(")",'')

df_con.drop('상장폐지사유',axis=1,inplace=True)

In [4]:
df_con.to_csv("../data/2. 연결개별통합.csv",index=False)